In [1]:
#26/8/24 Creates a K fold KNN model for each file in a folder provided they are padel

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        length = len(xtrain)
        nneigh = 100
        if length < nneigh:
            nneigh = length
        for mtry in range(1, nneigh):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = KNeighborsClassifier(n_neighbors=mtry, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = KNeighborsClassifier(n_neighbors=bestmetrics, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    finalresults.append(results[12])

For dataset Hepatotoxicity_PaDEL.csv
for fold 1 test set mcc of 0.3172115531668799 valid set mcc of 0.2013748963040803
for fold 2 test set mcc of 0.34040373505919186 valid set mcc of 0.350314365610564
for fold 3 test set mcc of 0.28769881081284937 valid set mcc of 0.1921261831344275
for fold 4 test set mcc of 0.35602429300237737 valid set mcc of 0.23538010330432038
for fold 5 test set mcc of 0.17216873355234694 valid set mcc of 0.23268946049775863

validation metrics of:
positives in data 249
negatives in data 226
net accuracy = 0.6463157894736842
mcc = 0.2889367465418612
For dataset NR-PPAR-gamma_PaDEL.csv
for fold 1 test set mcc of 0.21859641244629688 valid set mcc of 0.4078785175942527
for fold 2 test set mcc of 0.2261904761904762 valid set mcc of 0.5476190476190477
for fold 3 test set mcc of 0.7715167498104596 valid set mcc of 0.4615633996151572
for fold 4 test set mcc of 0.4113063728303113 valid set mcc of 0.08333333333333333
for fold 5 test set mcc of 0.5303300858899107 valid set

for fold 1 test set mcc of 0.1898957061123111 valid set mcc of 0.24645023077339334
for fold 2 test set mcc of 0.3418817293789138 valid set mcc of 0.23212021384556764
for fold 3 test set mcc of 0.47884305842642705 valid set mcc of 0.25677629550654774
for fold 4 test set mcc of 0.23251770418009515 valid set mcc of 0.38845420237659395
for fold 5 test set mcc of 0.2151483184233097 valid set mcc of -0.00528693668047267

validation metrics of:
positives in data 94
negatives in data 94
net accuracy = 0.5159574468085106
mcc = 0.03291540093383166
For dataset NR-ER_PaDEL.csv
for fold 1 test set mcc of 0.3601828617566437 valid set mcc of 0.30948334064246297
for fold 2 test set mcc of 0.44918470136732136 valid set mcc of 0.228578901794328
for fold 3 test set mcc of 0.5075566229399396 valid set mcc of 0.38062276248520116
for fold 4 test set mcc of 0.41637785158911667 valid set mcc of 0.4818718042486482
for fold 5 test set mcc of 0.2514886139456465 valid set mcc of 0.21445027340027564

validation me

In [5]:
finalresults

[0.5224392925598775,
 0.33743567470752406,
 0.4646785977740792,
 0.4516591109168654,
 0.5723387958404564,
 0.5589979253939279,
 0.381897590901659,
 0.41156557439955455,
 0.35349951843867855,
 0.37841890351674645,
 0.3926767941066129,
 0.3863095765811034,
 0.4442276861859421,
 0.3308801538385406,
 0.514525791312545,
 0.05976143046671968,
 0.3402804183494084,
 -0.16666666666666666,
 0.4077671491175933,
 0.38940111458138327,
 0.45383877172149034,
 0.05976143046671968,
 0.30000599502710085,
 0.5305539523742169,
 0.5720642066959932,
 0.23529652646066698,
 0.3773936184383908,
 0.4657214663500352,
 0,
 0.3510045885850323,
 0.5770485509757954,
 0.2869749231780359,
 0.3591556191769535,
 0.6073664063908583,
 0.5984209925969702,
 0.05976143046671968,
 0.5939723189065484,
 0.5602117964757503,
 0.4605346858287855,
 0.42556070567620496,
 0.6134162532192503,
 0.614948522161922,
 0.5274949660495826,
 0.2903943999526783,
 0.3275731085241172,
 0.45992667090925055,
 0.3071149881268544,
 0.452070985301534

In [2]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/PaDEL/data/padel_knn.csv')

In [14]:
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.300006
1,morganfp_SR-p53.csv,0.392793
2,morganfp_NR-AhR.csv,0.572339
3,morganfp_NR-ER-LBD.csv,0.453839
4,mutagenTTorsionfp.csv,0.514638
5,morganfp_NR-PPAR-gamma.csv,0.353500
6,Respiratory_ToxicityMorganfp.csv,0.421679
7,mutagenMACCSfp.csv,0.614949
8,Reproductive_ToxicityTTorsionfp.csv,0.000000
9,CarcinogenicityAvafp.csv,0.269629
